In [1]:
import transformers
import torch
from torch import nn
import pandas as pd
from transformers import pipeline
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
model_name = 'maxfil333/gpt2_imdb_generator_v2'

In [4]:
context = 'Well what can I say. Another adaptation of'

После обучения модели мы можем использовать ее, загрузив любым доступным способом. <br>
Далее показаны способы загрузки через <b>GPT2LMHeadModel</b> и <b>pipeline</b>

### GPT2LMHeadModel

In [5]:
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
generation_kwargs = {'do_sample': True,
                     'num_beams': 2,
                     'temperature': 1.5,
                     'top_p': 0.9,
                     'top_k': 30,
                     'max_length': 200,
                     'pad_token_id': tokenizer.eos_token_id}

In [7]:
input_ids = tokenizer.encode(context, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, **generation_kwargs)

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Well what can I say. Another adaptation of an old French film, this one starring Jean-Luc Godard and Claude-Michel Aufe, is very well made and entertaining. The plot and the characters are well drawn, and very well written. The film is quite well done, with plenty of humor, and very good acting and directing. I have seen a couple of times this film in the theaters, and it is very good to see it at home. The music in the film is excellent, and there is a good amount of dancing on the dance floor. The script is very well written, with very good dialogue. I really enjoyed seeing this film in the theaters. <br /><br />I highly recommend this film to a friend. It has a good story, and a good ensemble cast. The story is very good and well done. <br /><br />This movie is a great example of French Cinema. If you are in the market for a good French


### pipeline('text-generation')

In [8]:
pipe = pipeline('text-generation', model=model_name, tokenizer=model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
pipe(context, **generation_kwargs)

[{'generated_text': 'Well what can I say. Another adaptation of the novel. A great idea for a film. The plot and the plotline are very well executed. The cinematography and cinematography is very good. The acting is great. The acting is very good. This film is a great idea. I will recommend it to anyone who wants to watch it. It is very good. I am very glad that it was adapted from the novel, and I will be very happy to see it again. I do not think it is as good as the original. I think that the movie was written by a very good writer, who gave the novel a very good story. The acting and the plot line were very well executed. It is a very good idea to make a film with a good story. This is a very good idea for a film. It is very good. I will recommend it to anyone who wants to watch it. It is very good. I am very glad that it was adapted from the'}]

### comparison of default and pretrained model

In [10]:
default_generator_name = 'gpt2'
imdb_generator_name = model_name
generation_kwargs['max_length'] = 100
inputs = ["If you ask me",
          "I rarely pay attention",
          "Once upon a time",
          "So, I want to share my thoughts about",
          "No matter how sad it may sound"]

#### default gpt2

In [11]:
default_pipe = pipeline('text-generation', model=default_generator_name, tokenizer=default_generator_name)
default_outpus = default_pipe(inputs, **generation_kwargs)
default_outpus = [x[0]['generated_text'] for x in default_outpus]

#### pretrained on imdb gpt2

In [12]:
imdb_pipe = pipeline('text-generation', model=imdb_generator_name, tokenizer=imdb_generator_name)
imdb_outpus = imdb_pipe(inputs, **generation_kwargs)
imdb_outpus = [x[0]['generated_text'] for x in imdb_outpus]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### results:

In [13]:
default_outpus

['If you ask me, I\'m just saying it\'s not a big deal. It\'s something we\'re not going to talk about."\n\n"So what are you going to do?"\n\n"I\'m going to go into the kitchen. I\'m going to put the dishes in the oven. I\'m going to be in the kitchen, but I don\'t want to go into the oven. I want to go into the kitchen and get my breakfast. I don\'t want to go in',
 "I rarely pay attention to what people do with their money. I don't care. If they don't take care of themselves, they will die. It's not my place to say it, because I'm not a lawyer.\n\nI am a lawyer and I don't want to be one.\n\nI don't want to be the one who gets arrested because I'm a lawyer.\n\nI think about the things I do, and the things I don't do. I've",
 'Once upon a time, when it was not possible, he would ask himself, why do I want to die?"\n\n"Because I want to be loved. Because I want to be respected. Because I want to be loved. Because I want to be loved. Because I want to be loved. Because I want to be love

In [14]:
imdb_outpus

["If you ask me how the film ends, I'll tell you that the main character is a very intelligent, well-meaning and well-acted actor with a strong sense of humor. He's also very good at his job as a lawyer, so he gets the job done. He is the only character who doesn't fall into the trap of being the most stupid character in the movie. He's so good that I found myself laughing a lot at his antics, which is a shame, because he",
 "I rarely pay attention to this movie, and I don't know why. I don't know if it's because of the fact that I'm not a huge fan, or I just haven't seen the movie yet. It's just that it's so much more boring than the first time I watched it. The plot is boring, and the ending is boring. I feel like I'm giving it 2 out of 10. I didn't even watch it in the first place, so this movie is",
 'Once upon a time, there were a lot of movies that were based on real events. It was a good thing the genre could not get enough credit for that. There were a few movies that were base